<a href="https://colab.research.google.com/github/atta007/Speech-Recognizer/blob/main/speech_recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown https://storage.googleapis.com/google-code-archive-downloads/v2/code.google.com/hmm-speech-recognition/hmm-speech-recognition-0.1.zip

Downloading...
From: https://storage.googleapis.com/google-code-archive-downloads/v2/code.google.com/hmm-speech-recognition/hmm-speech-recognition-0.1.zip
To: /content/hmm-speech-recognition-0.1.zip
100% 857k/857k [00:00<00:00, 79.6MB/s]


In [ ]:
!unzip /content/hmm-speech-recognition-0.1.zip

In [ ]:
!pip install hmmlearn

     |████████████████████████████████| 374 kB 5.7 MB/s 


In [ ]:
!pip install winspeech

!pip install SpeechRecognition

ERROR: Could not find a version that satisfies the requirement pywin32 (from winspeech) (from versions: none)
ERROR: No matching distribution found for pywin32
     |████████████████████████████████| 32.8 MB 152 kB/s 


In [ ]:
!pip install SpeechRecognition

In [ ]:
!pip install speech-recognition

ERROR: Could not find a version that satisfies the requirement speech-recognition (from versions: none)
ERROR: No matching distribution found for speech-recognition


In [ ]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5888 sha256=1540aeb4285efae0c79447c106081542514f9fa71eb481e25c90b26d4326dd4d
  Stored in directory: /root/.cache/pip/wheels/b0/0e/94/28cd6afa3cd5998a63eef99fe31777acd7d758f59cf24839eb
Successfully built python-speech-features


In [ ]:
import os
import argparse
import warnings
import numpy as np
from scipy.io import wavfile
from hmmlearn import hmm
#from speech_features import mfcc
import speech_recognition 
from python_speech_features import mfcc


In [ ]:
# Define a class to train the HMM 
class ModelHMM(object):
    def __init__(self, num_components=4, num_iter=1000):
        self.n_components = num_components
        self.n_iter = num_iter

        self.cov_type = 'diag' 
        self.model_name = 'GaussianHMM' 

        self.models = []

        self.model = hmm.GaussianHMM(n_components=self.n_components, 
                covariance_type=self.cov_type, n_iter=self.n_iter)

    # 'training_data' is a 2D numpy array where each row is 13-dimensional
    def train(self, training_data):
        np.seterr(all='ignore')
        cur_model = self.model.fit(training_data)
        self.models.append(cur_model)

    # Run the HMM model for inference on input data
    def compute_score(self, input_data):
        return self.model.score(input_data)


In [ ]:
# Define a function to build a model for each word
def build_models(input_folder):
    # Initialize the variable to store all the models
    speech_models = []

    # Parse the input directory
    for dirname in os.listdir(input_folder):
        # Get the name of the subfolder 
        subfolder = os.path.join(input_folder, dirname)

        if not os.path.isdir(subfolder): 
            continue

        # Extract the label
        label = subfolder[subfolder.rfind('/') + 1:]

        # Initialize the variables
        X = np.array([])

        # Create a list of files to be used for training
        # We will leave one file per folder for testing
        training_files = [x for x in os.listdir(subfolder) if x.endswith('.wav')][:-1]

        # Iterate through the training files and build the models
        for filename in training_files: 
            # Extract the current filepath
            filepath = os.path.join(subfolder, filename)

            # Read the audio signal from the input file
            sampling_freq, signal = wavfile.read(filepath)
            
            # Extract the MFCC features
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                features_mfcc = mfcc(signal, sampling_freq)

            # Append to the variable X
            if len(X) == 0:
                X = features_mfcc
            else:
                X = np.append(X, features_mfcc, axis=0)
            
        # Create the HMM model
        model = ModelHMM()

        # Train the HMM
        model.train(X)

        # Save the model for the current word
        speech_models.append((model, label))

        # Reset the variable
        model = None

    return speech_models

# Define a function to run tests on input files
def run_tests(test_files):
    # Classify input data
    for test_file in test_files:
        # Read input file
        sampling_freq, signal = wavfile.read(test_file)

        # Extract MFCC features
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            features_mfcc = mfcc(signal, sampling_freq)

        # Define variables
        max_score = -float('inf') 
        output_label = None 

        # Run the current feature vector through all the HMM
        # models and pick the one with the highest score
        for item in speech_models:
            model, label = item
            score = model.compute_score(features_mfcc)
            if score > max_score:
                max_score = score
                predicted_label = label

        # Print the predicted output 
        start_index = test_file.find('/') + 1
        end_index = test_file.rfind('/')
        original_label = test_file[start_index:end_index]
        print('\nOriginal: ', original_label) 
        print('Predicted:', predicted_label)

if __name__=='__main__':
    #args = build_arg_parser().parse_args()
    input_folder = '/content/hmm-speech-recognition-0.1/audio'

    # Build an HMM model for each word
    speech_models = build_models(input_folder)

    # Test files -- the 15th file in each subfolder 
    test_files = []
    for root, dirs, files in os.walk(input_folder):
        for filename in (x for x in files if '15' in x):
            filepath = os.path.join(root, filename)
            test_files.append(filepath)

    run_tests(test_files)


Original:  content/hmm-speech-recognition-0.1/audio/lime
Predicted: lime

Original:  content/hmm-speech-recognition-0.1/audio/kiwi
Predicted: kiwi

Original:  content/hmm-speech-recognition-0.1/audio/orange
Predicted: orange

Original:  content/hmm-speech-recognition-0.1/audio/apple
Predicted: apple

Original:  content/hmm-speech-recognition-0.1/audio/peach
Predicted: peach

Original:  content/hmm-speech-recognition-0.1/audio/banana
Predicted: banana

Original:  content/hmm-speech-recognition-0.1/audio/pineapple
Predicted: pineapple
